In [3]:
import numpy as np
# First, let's load the provided dataset to understand its structure and content.
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
file_path = r'../Dataset/SurveyData/SurveyData_CSV_DAP_2021_HLES_dog_owner_v1.0.csv'
dog_health_data = pd.read_csv(file_path, delimiter=',')  # Assuming tab-delimited format based on the file extension
dog_health = pd.read_csv(file_path, delimiter=',')

dog_health_data.head()

/var/folders/zd/3_6frj8s08b9zb1f3y0xqr2r0000gn/T/ipykernel_7002/3636265617.py:8: DtypeWarning: Columns (5,21,75,77,79,81,83,85,87,89,191,192,193,194,195,196,197,198,199,200,201,204,214,285,295,516,566) have mixed types. Specify dtype option on import or set low_memory=False.
  dog_health_data = pd.read_csv(file_path, delimiter=',')  # Assuming tab-delimited format based on the file extension
/var/folders/zd/3_6frj8s08b9zb1f3y0xqr2r0000gn/T/ipykernel_7002/3636265617.py:9: DtypeWarning: Columns (5,21,75,77,79,81,83,85,87,89,191,192,193,194,195,196,197,198,199,200,201,204,214,285,295,516,566) have mixed types. Specify dtype option on import or set low_memory=False.
  dog_health = pd.read_csv(file_path, delimiter=',')


,dog_id,owner_id,st_hles_completion_date,dd_breed_pure_or_mixed,dd_breed_pure,dd_breed_pure_non_akc,dd_breed_mixed_primary,dd_breed_mixed_secondary,dd_age_years,dd_age_basis,...,oc_primary_residence_state,oc_primary_residence_census_division,oc_primary_residence_ownership,oc_primary_residence_ownership_other_description,oc_primary_residence_time_percentage,oc_secondary_residence,oc_secondary_residence_state,oc_secondary_residence_ownership,oc_secondary_residence_ownership_other_description,oc_secondary_residence_time_percentage
0,10509,14660,2020-10-17,2,NaN,NaN,16.0,999.0,16.333333,2,...,MO,4.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
1,74227,16755,2020-10-26,2,NaN,NaN,133.0,12.0,1.833333,1,...,MD,5.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
2,32487,8568,2020-09-07,1,159.0,NaN,NaN,NaN,13.000000,3,...,VA,5.0,1,NaN,NaN,False,NaN,NaN,NaN,NaN
3,63150,5380,2020-08-21,2,NaN,NaN,999.0,999.0,13.000000,3,...,CA,9.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN
4,33369,15957,2020-10-23,2,NaN,NaN,230.0,82.0,2.166667,1,...,OH,3.0,2,NaN,NaN,False,NaN,NaN,NaN,NaN


In [4]:
columns_to_preserve = ['df_feedings_per_day', 'df_diet_consistency', 
                       'df_infrequent_supplements', 'df_daily_supplements', 
                       'df_primary_diet_component_organic', 'df_primary_diet_component_grain_free']

# 选择需要统计的列
columns_to_count = dog_health[columns_to_preserve]

# 统计每个值的数量
counts_dict = {}

for column in columns_to_count:
    counts = dog_health[column].value_counts()
    counts_dict[column] = counts

# 输出结果
for column, counts in counts_dict.items():
    print(f"Column: {column}")
    print(counts)
    print("\n")


Column: df_feedings_per_day
df_feedings_per_day
2    24421
4     3533
3     2715
1     2503
Name: count, dtype: int64


Column: df_diet_consistency
df_diet_consistency
1     29601
2      3312
3       228
98       31
Name: count, dtype: int64


Column: df_infrequent_supplements
df_infrequent_supplements
False    29553
True      3619
Name: count, dtype: int64


Column: df_daily_supplements
df_daily_supplements
False    18089
True     15083
Name: count, dtype: int64


Column: df_primary_diet_component_organic
df_primary_diet_component_organic
False    26576
True      6596
Name: count, dtype: int64


Column: df_primary_diet_component_grain_free
df_primary_diet_component_grain_free
False    19492
True     13680
Name: count, dtype: int64




In [5]:
#dog_health['df_infrequent_supplements'].value_counts()
dog_health['df_supplements'] = ~(dog_health['df_daily_supplements'] | dog_health['df_infrequent_supplements'])
dog_health['df_supplements'].value_counts()

df_supplements
False    17029
True     16143
Name: count, dtype: int64

In [6]:
# Merge 'df_diet_consistency' and 'df_infrequent_supplements' into a new column 'df_supplements'
dog_health['df_supplements'] = ~(dog_health['df_daily_supplements'] | dog_health['df_infrequent_supplements'])

filtered_dog_health = dog_health[['dog_id', 'df_feedings_per_day', 'df_diet_consistency',
                       'df_supplements', 'df_primary_diet_component_organic', 'df_primary_diet_component_grain_free']]

# Replace values of 98 with np.nan in 'df_diet_consistency'
filtered_dog_health['df_diet_consistency'] = filtered_dog_health['df_diet_consistency'].replace(98, np.nan)

# Columns to preserve
columns_to_preserve = ['dog_id', 'df_feedings_per_day', 'df_diet_consistency',
                       'df_supplements', 'df_primary_diet_component_organic', 'df_primary_diet_component_grain_free']

# Save the new DataFrame to a CSV file
#filtered_dog_health.to_csv('one_hot_encoded_diet_input.csv', index=False)

# Display the new DataFrame
filtered_dog_health

/var/folders/zd/3_6frj8s08b9zb1f3y0xqr2r0000gn/T/ipykernel_7002/239063219.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dog_health['df_diet_consistency'] = filtered_dog_health['df_diet_consistency'].replace(98, np.nan)


,dog_id,df_feedings_per_day,df_diet_consistency,df_supplements,df_primary_diet_component_organic,df_primary_diet_component_grain_free
0,10509,2,1.0,True,False,True
1,74227,2,1.0,True,False,True
2,32487,2,1.0,False,False,True
3,63150,2,1.0,False,True,True
4,33369,2,1.0,True,False,False
...,...,...,...,...,...,...
33167,91849,2,1.0,True,False,False
33168,33783,2,1.0,True,True,True
33169,98862,2,1.0,True,False,False
33170,99462,2,1.0,False,False,True


In [7]:
filtered_dog_health['df_primary_diet_component_grain_free'].value_counts()

df_primary_diet_component_grain_free
False    19492
True     13680
Name: count, dtype: int64

In [8]:
filtered_dog_health = filtered_dog_health[filtered_dog_health['df_diet_consistency'].notna()]

In [9]:
# Normalize the specified features in one line and save in the same columns
#features_to_normalize = ['df_feedings_per_day', 'df_diet_consistency']

# Initialize MinMaxScaler
#scaler = MinMaxScaler()

# Normalize
#filtered_dog_health[features_to_normalize] = scaler.fit_transform(filtered_dog_health[features_to_normalize])
#filtered_dog_health

In [10]:
# make a new pickle file can load by our model
filtered_dog_health.to_pickle('features/diet.pkl')
filtered_dog_health.to_csv('features/diet.csv', index=False)